In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
import re, os, joblib, json
from tqdm.notebook import tqdm

#  Barre de progression pour apply
tqdm.pandas()

# === 0. Création du dossier final s'il n'existe pas
final_dir = "/content/drive/MyDrive/Colab Notebooks/fahim/final"
os.makedirs(final_dir, exist_ok=True)
print(f" Dossier créé ou déjà existant : {final_dir}")

# === 1. Chargement des données de base
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/X_train_fr.csv")

# === 2. Nettoyage : on garde uniquement les lignes traduites + chemin image non vide
df = df[df["translated_text_combined"].notna()]
df = df[df["image_path"].notna() & df["image_path"].str.strip().ne("")].copy()

# === 3. Nettoyage HTML si nécessaire
def maybe_clean_html(text):
    if bool(re.search(r"<[^>]+>|&\w+;", text)):  # détecte balises ou entités HTML
        return BeautifulSoup(str(text), "html.parser").get_text()
    return text

print(" Nettoyage HTML en cours...")
df["clean_text"] = df["translated_text_combined"].progress_apply(maybe_clean_html)

# === 4. Rééquilibrage des classes (2000 à 5000 lignes par prdtypecode)
min_samples = 2000
max_samples = 5000
balanced = []

print("⚖️ Rééquilibrage des classes...")
for cat in tqdm(df["prdtypecode"].unique(), desc="Catégories"):
    sub = df[df["prdtypecode"] == cat]
    if len(sub) < min_samples:
        sub = resample(sub, replace=True, n_samples=min_samples, random_state=42)
    elif len(sub) > max_samples:
        sub = resample(sub, replace=False, n_samples=max_samples, random_state=42)
    balanced.append(sub)

df_balanced = pd.concat(balanced).reset_index(drop=True)

# === 5. Encodage du label
le = LabelEncoder()
df_balanced["label"] = le.fit_transform(df_balanced["prdtypecode"])

# === 6. Sauvegardes dans le dossier "final"
save_csv = os.path.join(final_dir, "X_trainfr_final.csv")
save_encoder = os.path.join(final_dir, "label_encoder_final.pkl")
save_json = os.path.join(final_dir, "label_mapping_final.json")

df_balanced.to_csv(save_csv, index=False)
joblib.dump(le, save_encoder)

# === 7. Mapping lisible
prdtypecode_to_label = {
    10: "Livres", 40: "Jeux Vidéo", 50: "Accessoires jeux vidéos", 60: "Jeux vidéo & Consoles",
    1140: "Figurines", 1160: "Cartes de jeux", 1180: "Jeux de rôle et de figurines", 1280: "Jouets & Enfant",
    1281: "Jeux de société", 1300: "Véhicules RC & miniatures", 1301: "Chaussettes bébé", 1302: "Sports & Loisirs",
    1320: "Puériculture", 1560: "Maison", 1920: "Linge de maison", 1940: "Petit déjeuner", 2060: "Décoration",
    2220: "Animalerie", 2280: "Revue", 2403: "Lots Livres & Revues", 2462: "Lots consoles & jeux",
    2522: "Fournitures Papeterie", 2582: "Mobilier de jardin", 2583: "Équipement piscine & spa",
    2585: "Outillage de jardin", 2705: "eBooks", 2905: "Jeux en téléchargement"
}

mapping = {
    str(i): {
        "prdtypecode": int(code),
        "label_name": prdtypecode_to_label[int(code)]
    }
    for i, code in enumerate(le.classes_)
}

with open(save_json, "w") as f:
    json.dump(mapping, f, indent=4, ensure_ascii=False)

# === 8. Logs finaux
print(f"\n Fichier final : {save_csv}")
print(f" LabelEncoder : {save_encoder}")
print(f" Mapping JSON : {save_json}")
print("\n Répartition des classes :")
print(df_balanced["prdtypecode"].value_counts())


 Dossier créé ou déjà existant : /content/drive/MyDrive/Colab Notebooks/fahim/final
 Nettoyage HTML en cours...


  0%|          | 0/84916 [00:00<?, ?it/s]

⚖️ Rééquilibrage des classes...


Catégories:   0%|          | 0/27 [00:00<?, ?it/s]


 Fichier final : /content/drive/MyDrive/Colab Notebooks/fahim/final/X_trainfr_final.csv
 LabelEncoder : /content/drive/MyDrive/Colab Notebooks/fahim/final/label_encoder_final.pkl
 Mapping JSON : /content/drive/MyDrive/Colab Notebooks/fahim/final/label_mapping_final.json

 Répartition des classes :
prdtypecode
1560    5000
1300    5000
2583    5000
2060    4993
2522    4989
1280    4870
2403    4774
2280    4760
1920    4303
1160    3953
1320    3241
10      3116
2705    2761
1140    2671
2582    2589
40      2508
2585    2496
1302    2491
1281    2070
50      2000
1180    2000
2462    2000
60      2000
2220    2000
2905    2000
1940    2000
1301    2000
Name: count, dtype: int64
